In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData, UVCal, UVFlag, utils
from glob import glob
from astropy import units as u
from astropy import constants as c
from astropy.time import Time
from astropy.coordinates import get_body, SkyCoord, AltAz, EarthLocation, Angle
from copy import deepcopy

In [2]:
JD = '2457548'
datadir = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/'

In [3]:
files = glob(datadir+'zen.'+JD+'**'+'uvcRP.uvh5')

In [4]:
%%time

#this script writes out uvflag objects 

for file in files: 
    #prep data before making different types of flags 
    uvd = UVData()
    uvd.read(file)
    uvf = UVFlag(uvd, mode='flag', copy_flags=True, run_check=True)
    uvf_or = deepcopy(uvf)
    uvf_thresh = deepcopy(uvf)
    uvf_greedy = deepcopy(uvf)
    
    #uvf_or
    uvf_or.to_waterfall(method='or', keep_pol=False)
    #uvf_thresh
    uvf_thresh.to_waterfall(method='mean', keep_pol=False)
    #uvf_greedy 
    uvf_greedy.to_waterfall(method='or', keep_pol=False)
    time_frac = uvf_greedy.flag_array.mean(axis=1).squeeze()
    freq_frac = uvf_greedy.flag_array.mean(axis=0).squeeze()
    uvf_greedy.flag_array[time_frac>0.33,:,0] = True
    uvf_greedy.flag_array[:,freq_frac>0.10,0] = True
    
    #prep for writing out, getting JD snd unique lst for each
    filename = file.split('/')[-1]
    tmp = filename.split('.')
    juldate = tmp[1]+'.'+tmp[2]
    outpath_or = '/lustre/aoc/projects/hera/agavidia/TimeVariability/HERA19Data/flags_or/'
    outpath_thresh = '/lustre/aoc/projects/hera/agavidia/TimeVariability/HERA19Data/flags_thresh//'
    outpath_greedy = '/lustre/aoc/projects/hera/agavidia/TimeVariability/HERA19Data/flags_greedy/'
    
    #read out uvf object
    uvf_or.write(outpath_or+'flag.'+juldate+'_or.uvh5', clobber=True)
    uvf_thresh.write(outpath_thresh+'flag.'+juldate+'_thresh.uvh5', clobber=True)
    uvf_greedy.write(outpath_greedy+'flag.'+juldate+'_greedy.uvh5', clobber=True)
    
    
    #applying the flags to the data...i think i need to make copies of the uvd..?
    #uvd_or = deepcopy(uvd)
    #uvd_thresh = deepcopy(uvd)
    #uvd_greedy = deepcopy(uvd)
    #utils.apply_uvflag(uvd_or, uvf_or)
    #utils.apply_uvflag(uvd_thresh, uvf_thresh)  #can only apply when it is in flag mode, not metric?
    #utils.apply_uvflag(uvd_greedy, uvf_greedy)
    
    #write out the uvd
    #uvd_or.write_uvh5(outpath_or+'flag.'+juldate+'_or.uvh5')
    #uvd_thresh.write_uvh5(outpath_thresh+'flag.'+juldate+'_thresh.uvh5')
    #uvd_greedy.write_uvh5(outpath_greedy+'flag.'+juldate+'_greedy.uvh5')

CPU times: user 3min 57s, sys: 1min 36s, total: 5min 34s
Wall time: 7min 56s
